# Lab 1 - CFPL
Team Members:
- Achchala Deepan - 20943939
- Manjary Muruganandan - 20950662

In [2]:
from cplex import Cplex
from docplex.mp.model import Model

In [3]:
facilities = ['F1', 'F2', 'F3']
customers = ['C1', 'C2', 'C3', 'C4']

setup_cost = {
    'F1': 1000,
    'F2': 1200,
    'F3': 1100
}

capacity = {
    'F1': 50,
    'F2': 60,
    'F3': 40
}

demand = {
    'C1': 10,
    'C2': 20,
    'C3': 15,
    'C4': 25
}

distance = {
    ('F1', 'C1'): 4, ('F1', 'C2'): 6, ('F1', 'C3'): 9, ('F1', 'C4'): 5,
    ('F2', 'C1'): 5, ('F2', 'C2'): 4, ('F2', 'C3'): 7, ('F2', 'C4'): 6,
    ('F3', 'C1'): 6, ('F3', 'C2'): 3, ('F3', 'C3'): 4, ('F3', 'C4'): 7,
}

mdl = Model(name="Mini_CFLP")

open_f = mdl.binary_var_dict(facilities, name='Open')
ship = mdl.continuous_var_dict([(f, c) for f in facilities for c in customers], name='Ship')

for c in customers:
    mdl.add_constraint(mdl.sum(ship[f, c] for f in facilities) == demand[c], f"Demand_{c}")

for f in facilities:
    mdl.add_constraint(mdl.sum(ship[f, c] for c in customers) <= capacity[f] * open_f[f], f"Capacity_{f}")


setup_total = mdl.sum(setup_cost[f] * open_f[f] for f in facilities)
transport_total = mdl.sum(ship[f, c] * distance[f, c] for f in facilities for c in customers)

mdl.minimize(setup_total + transport_total)

solution = mdl.solve(log_output=True)

if solution:
    print("Total Cost:", mdl.objective_value)
    print("\nFacilities Opened:")
    for f in facilities:
        if open_f[f].solution_value > 0.5:
            print(f"  {f}")
    print("\nShipping Plan:")
    for f in facilities:
        for c in customers:
            shipped = ship[f, c].solution_value
            if shipped > 0.01:
                print(f"  {f} → {c}: {shipped} units")
else:
    print("No feasible solution found.")

Version identifier: 22.1.1.0 | 2022-11-28 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
Reduced MIP has 7 rows, 15 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Found incumbent of value 3630.000000 after 0.01 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
Reduced MIP has 7 rows, 15 columns, and 27 nonzeros.
Reduced MIP has 3 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Probing time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.00 sec. (0.01 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         3630.0000      